In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Stroke"
cohort = "GSE58294"

# Input paths
in_trait_dir = "../../input/GEO/Stroke"
in_cohort_dir = "../../input/GEO/Stroke/GSE58294"

# Output paths
out_data_file = "../../output/preprocess/Stroke/GSE58294.csv"
out_gene_data_file = "../../output/preprocess/Stroke/gene_data/GSE58294.csv"
out_clinical_data_file = "../../output/preprocess/Stroke/clinical_data/GSE58294.csv"
json_path = "../../output/preprocess/Stroke/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Gene Expression Following Cardioembolic Stroke"
!Series_summary	"Blood from subjects with cardioembolic stroke and controls was collected, and the RNA extracted was interrogated and whole genome U133 Affymetrix Arrays. Twenty-three control samples and sixty-nine cardioembolic stroke samples were assayed."
!Series_overall_design	"Blood from subjects with cardioembolic stroke and controls was collected, and the RNA extracted was interrogated and whole genome U133 Affymetrix Arrays. Twenty-three control samples and sixty-nine cardioembolic stroke samples were assayed. Cardioembolic stroke subjects were analyzed at three time points: less than 3 hours, 5 hours, and 24 hours following the event."
Sample Characteristics Dictionary:
{0: ['group: Control', 'group: Cardioembolic Stroke'], 1: ['time after stroke (h): Control', 'time after stroke (h): 3', 'time after stroke (h): 5', 'time after stroke (h): 24'], 2: ['subject id: Control-015', 'subject id: Co

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# From the background information, this is a whole genome Affymetrix Arrays study,
# which indicates gene expression data is available
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# Examining the sample characteristics dictionary:
# - For trait: Key 0 has 'group: Control' and 'group: Cardioembolic Stroke' which indicates stroke status
# - For age: No explicit age information in the sample characteristics 
# - For gender: No explicit gender information in the sample characteristics
trait_row = 0  # Trait data is available under key 0
age_row = None  # Age data is not available
gender_row = None  # Gender data is not available

# 2.2 Data Type Conversion Functions
def convert_trait(value):
    """Convert trait value to binary: 1 for stroke, 0 for control."""
    if value is None:
        return None
    # Extract the value after colon if present
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    if "control" in value.lower():
        return 0
    elif "stroke" in value.lower() or "cardioembolic" in value.lower():
        return 1
    else:
        return None

def convert_age(value):
    """Convert age value to continuous numeric format."""
    # This function is defined but not used since age data is unavailable
    if value is None:
        return None
    
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    try:
        return float(value)
    except:
        return None

def convert_gender(value):
    """Convert gender value to binary: 0 for female, 1 for male."""
    # This function is defined but not used since gender data is unavailable
    if value is None:
        return None
    
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    if value.lower() in ["female", "f"]:
        return 0
    elif value.lower() in ["male", "m"]:
        return 1
    else:
        return None

# 3. Save Metadata
# Initial filtering: trait data is available since trait_row is not None
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Trait data is available, so we proceed with extraction
if trait_row is not None:
    # Check if 'clinical_data' variable exists in the environment first
    try:
        # Check if clinical_data is already defined
        clinical_data
    except NameError:
        # If not defined, we need to load it or inform that it's missing
        print("Clinical data not found in the environment.")
        # We'll use a workaround by loading the matrices file which should contain the clinical data
        try:
            matrices_file = glob.glob(os.path.join(in_cohort_dir, "*_series_matrix.txt*"))
            if matrices_file:
                # Read the clinical data from the series matrix file
                clinical_data = pd.read_csv(matrices_file[0], sep='\t', comment='!', nrows=100)
            else:
                raise FileNotFoundError("Series matrix file not found.")
        except Exception as e:
            print(f"Error loading clinical data: {e}")
            # Since we can't proceed without clinical data, we'll exit this section
            print("Skipping clinical feature extraction due to missing data.")
            exit_clinical_feature_extraction = True
    
    # Only proceed if we have valid clinical data
    if 'exit_clinical_feature_extraction' not in locals():
        # Extract clinical features
        selected_clinical_df = geo_select_clinical_features(
            clinical_df=clinical_data,
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            age_row=age_row,
            convert_age=convert_age,
            gender_row=gender_row,
            convert_gender=convert_gender
        )
        
        # Preview the extracted clinical data
        print("Preview of extracted clinical features:")
        print(preview_df(selected_clinical_df))
        
        # Save the clinical data
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        selected_clinical_df.to_csv(out_clinical_data_file, index=False)
        print(f"Clinical data saved to {out_clinical_data_file}")


Preview of extracted clinical features:
{'GSM1406033': [0.0], 'GSM1406034': [0.0], 'GSM1406035': [0.0], 'GSM1406036': [0.0], 'GSM1406037': [0.0], 'GSM1406038': [0.0], 'GSM1406039': [0.0], 'GSM1406040': [0.0], 'GSM1406041': [0.0], 'GSM1406042': [0.0], 'GSM1406043': [0.0], 'GSM1406044': [0.0], 'GSM1406045': [0.0], 'GSM1406046': [0.0], 'GSM1406047': [0.0], 'GSM1406048': [0.0], 'GSM1406049': [0.0], 'GSM1406050': [0.0], 'GSM1406051': [0.0], 'GSM1406052': [0.0], 'GSM1406053': [0.0], 'GSM1406054': [0.0], 'GSM1406055': [0.0], 'GSM1406056': [1.0], 'GSM1406057': [1.0], 'GSM1406058': [1.0], 'GSM1406059': [1.0], 'GSM1406060': [1.0], 'GSM1406061': [1.0], 'GSM1406062': [1.0], 'GSM1406063': [1.0], 'GSM1406064': [1.0], 'GSM1406065': [1.0], 'GSM1406066': [1.0], 'GSM1406067': [1.0], 'GSM1406068': [1.0], 'GSM1406069': [1.0], 'GSM1406070': [1.0], 'GSM1406071': [1.0], 'GSM1406072': [1.0], 'GSM1406073': [1.0], 'GSM1406074': [1.0], 'GSM1406075': [1.0], 'GSM1406076': [1.0], 'GSM1406077': [1.0], 'GSM1406078': 

### Step 3: Gene Data Extraction

In [4]:
# 1. Get the SOFT and matrix file paths again 
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"Matrix file found: {matrix_file}")

# 2. Use the get_genetic_data function from the library to get the gene_data
try:
    gene_data = get_genetic_data(matrix_file)
    print(f"Gene data shape: {gene_data.shape}")
    
    # 3. Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")


Matrix file found: ../../input/GEO/Stroke/GSE58294/GSE58294_series_matrix.txt.gz


Gene data shape: (54675, 92)
First 20 gene/probe identifiers:
Index(['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at', '1294_at',
       '1316_at', '1320_at', '1405_i_at', '1431_at', '1438_at', '1487_at',
       '1494_f_at', '1552256_a_at', '1552257_a_at', '1552258_at', '1552261_at',
       '1552263_at', '1552264_a_at', '1552266_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Examining the gene identifiers from the previous output
# These identifiers (e.g., "1007_s_at") are Affymetrix probe IDs, not human gene symbols
# They follow the format of Affymetrix microarray probe identifiers which need to be mapped to gene symbols

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Analyze the gene annotation dataframe to identify which columns contain the gene identifiers and gene symbols
print("\nGene annotation preview:")
print(f"Columns in gene annotation: {gene_annotation.columns.tolist()}")
print(preview_df(gene_annotation, n=5))

# Let's look for platform information in the SOFT file to understand the annotation better
print("\nSearching for platform information in SOFT file:")
with gzip.open(soft_file, 'rt') as f:
    for i, line in enumerate(f):
        if '!Series_platform_id' in line:
            print(line.strip())
            break
        if i > 100:  # Limit search to first 100 lines
            print("Platform ID not found in first 100 lines")
            break

# Check if the SOFT file includes any reference to gene symbols
print("\nSearching for gene symbol information in SOFT file:")
with gzip.open(soft_file, 'rt') as f:
    gene_symbol_lines = []
    for i, line in enumerate(f):
        if 'GENE_SYMBOL' in line or 'gene_symbol' in line.lower() or 'symbol' in line.lower():
            gene_symbol_lines.append(line.strip())
        if i > 1000 and len(gene_symbol_lines) > 0:  # Limit search but ensure we found something
            break
    
    if gene_symbol_lines:
        print("Found references to gene symbols:")
        for line in gene_symbol_lines[:5]:  # Show just first 5 matches
            print(line)
    else:
        print("No explicit gene symbol references found in first 1000 lines")

# Look for alternative annotation files or references in the directory
print("\nChecking for additional annotation files in the directory:")
all_files = os.listdir(in_cohort_dir)
print([f for f in all_files if 'annotation' in f.lower() or 'platform' in f.lower() or 'gpl' in f.lower()])



Gene annotation preview:
Columns in gene annotation: ['ID', 'GB_ACC', 'SPOT_ID', 'Species Scientific Name', 'Annotation Date', 'Sequence Type', 'Sequence Source', 'Target Description', 'Representative Public ID', 'Gene Title', 'Gene Symbol', 'ENTREZ_GENE_ID', 'RefSeq Transcript ID', 'Gene Ontology Biological Process', 'Gene Ontology Cellular Component', 'Gene Ontology Molecular Function']
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database', 'GenBank', 

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify which columns contain probe IDs and gene symbols
# From the gene annotation preview, we can see:
# - 'ID' column contains probe IDs (e.g., "1007_s_at")
# - 'Gene Symbol' column contains gene symbols (e.g., "DDR1 /// MIR4640")

# Extract the mapping between probe IDs and gene symbols
mapping_df = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='Gene Symbol')
print(f"Mapping dataframe shape: {mapping_df.shape}")
print("First few rows of mapping dataframe:")
print(mapping_df.head())

# Apply the gene mapping to convert probe-level measurements to gene expression data
gene_data = apply_gene_mapping(gene_data, mapping_df)
print(f"Transformed gene data shape: {gene_data.shape}")
print("First 10 gene symbols in transformed gene data:")
print(gene_data.index[:10])

# Save the gene expression data to the output file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


Mapping dataframe shape: (45782, 2)
First few rows of mapping dataframe:
          ID              Gene
0  1007_s_at  DDR1 /// MIR4640
1    1053_at              RFC2
2     117_at             HSPA6
3     121_at              PAX8
4  1255_g_at            GUCA1A
Transformed gene data shape: (21278, 92)
First 10 gene symbols in transformed gene data:
Index(['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2M-AS1', 'A2ML1', 'A2MP1',
       'A4GALT', 'A4GNT', 'AA06'],
      dtype='object', name='Gene')


Gene expression data saved to ../../output/preprocess/Stroke/gene_data/GSE58294.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the obtained gene expression data
# Use the gene_data variable that was already loaded in Step 4
print(f"Original gene data shape: {gene_data.shape}")

# Normalize gene symbols
gene_data_normalized = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape after normalization: {gene_data_normalized.shape}")

# Save the normalized gene expression data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data_normalized.to_csv(out_gene_data_file)
print(f"Normalized gene expression data saved to {out_gene_data_file}")

# 2. Load the clinical data that was saved in Step 2
clinical_data_file = out_clinical_data_file
if os.path.exists(clinical_data_file):
    clinical_data = pd.read_csv(clinical_data_file)
    print(f"Clinical data loaded from {clinical_data_file}, shape: {clinical_data.shape}")
    
    # Print columns for debugging
    print("Clinical data columns:", clinical_data.columns.tolist())
    
    # Check if there are any actual trait columns
    if len(clinical_data.columns) > 1:  # Has more than just index column
        # Set the first column as index if it's not already
        if not clinical_data.index.name:
            clinical_data.set_index(clinical_data.columns[0], inplace=True)
        
        # Link clinical and genetic data
        linked_data = geo_link_clinical_genetic_data(clinical_data, gene_data_normalized)
        print(f"Linked data shape: {linked_data.shape}")
        print("Linked data columns (first 10):", linked_data.columns[:10].tolist())
        
        # Create a temporary column with the trait name for compatibility with handle_missing_values
        if trait not in linked_data.columns:
            # Assuming the first column after the index is the trait column
            first_col = linked_data.columns[0]
            linked_data[trait] = linked_data[first_col]
            print(f"Created '{trait}' column from '{first_col}' for compatibility")
        
        # 3. Handle missing values in the linked data
        linked_data = handle_missing_values(linked_data, trait)
        print(f"Data shape after handling missing values: {linked_data.shape}")
        
        # 4. Determine whether the trait and some demographic features in the dataset are severely biased
        is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
        
        # 5. Conduct final quality validation
        is_usable = validate_and_save_cohort_info(
            is_final=True,
            cohort=cohort,
            info_path=json_path,
            is_gene_available=True,
            is_trait_available=True,
            is_biased=is_biased,
            df=linked_data,
            note="Dataset contains gene expression data from peripheral blood of stroke patients and controls."
        )
        
        # 6. If the linked data is usable, save it
        if is_usable:
            os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
            linked_data.to_csv(out_data_file)
            print(f"Linked data saved to {out_data_file}")
        else:
            print("Dataset not usable due to severe bias. Not saving linked data.")
    else:
        print("Clinical data has insufficient columns.")
        
        # Create a minimal dataframe for validation purposes
        empty_df = pd.DataFrame({"dummy": [0]})
        
        # Mark as not usable
        is_usable = validate_and_save_cohort_info(
            is_final=True,
            cohort=cohort,
            info_path=json_path,
            is_gene_available=True,
            is_trait_available=False,
            is_biased=True,
            df=empty_df,
            note="Clinical data has insufficient columns for proper analysis."
        )
else:
    print(f"Clinical data file {clinical_data_file} not found.")
    
    # Create a minimal dataframe for validation purposes
    empty_df = pd.DataFrame({"dummy": [0]})
    
    # Mark as not usable
    is_usable = validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=True,
        is_trait_available=False,
        is_biased=True,
        df=empty_df,
        note="Clinical data file not found, despite earlier extraction. Dataset cannot be processed completely."
    )

Original gene data shape: (21278, 92)
Gene data shape after normalization: (19845, 92)


Normalized gene expression data saved to ../../output/preprocess/Stroke/gene_data/GSE58294.csv
Clinical data loaded from ../../output/preprocess/Stroke/clinical_data/GSE58294.csv, shape: (1, 92)
Clinical data columns: ['GSM1406033', 'GSM1406034', 'GSM1406035', 'GSM1406036', 'GSM1406037', 'GSM1406038', 'GSM1406039', 'GSM1406040', 'GSM1406041', 'GSM1406042', 'GSM1406043', 'GSM1406044', 'GSM1406045', 'GSM1406046', 'GSM1406047', 'GSM1406048', 'GSM1406049', 'GSM1406050', 'GSM1406051', 'GSM1406052', 'GSM1406053', 'GSM1406054', 'GSM1406055', 'GSM1406056', 'GSM1406057', 'GSM1406058', 'GSM1406059', 'GSM1406060', 'GSM1406061', 'GSM1406062', 'GSM1406063', 'GSM1406064', 'GSM1406065', 'GSM1406066', 'GSM1406067', 'GSM1406068', 'GSM1406069', 'GSM1406070', 'GSM1406071', 'GSM1406072', 'GSM1406073', 'GSM1406074', 'GSM1406075', 'GSM1406076', 'GSM1406077', 'GSM1406078', 'GSM1406079', 'GSM1406080', 'GSM1406081', 'GSM1406082', 'GSM1406083', 'GSM1406084', 'GSM1406085', 'GSM1406086', 'GSM1406087', 'GSM1406088

Data shape after handling missing values: (91, 19847)
For the feature 'Stroke', the least common label is '0.0' with 22 occurrences. This represents 24.18% of the dataset.
The distribution of the feature 'Stroke' in this dataset is fine.



Linked data saved to ../../output/preprocess/Stroke/GSE58294.csv
